In [4]:
library(tidyverse)
library(lubridate)
library(forecast)
library(rnoaa)

Warning message:
"package 'rnoaa' was built under R version 4.1.2"
Registered S3 method overwritten by 'hoardr':
  method           from
  print.cache_info httr



In [5]:
stations <- ghcnd_stations()

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/ghcnd-stations.rds

date created (size, mb): 2022-02-12 20:25:20 (2.159)

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/ghcnd-inventory.rds

date created (size, mb): 2022-02-12 20:26:28 (2.669)



In [17]:
kyoto_extra <- read.csv("processed/kyoto_extra.csv")
liestal_extra <- read.csv("processed/liestal_extra.csv")
dc_extra <- read.csv("processed/dc_extra.csv")
van_extra <- read.csv("processed/van_extra.csv")


In [188]:
kyoto_weather <- ghcnd_search(stationid = "JA000047759", date_min = "2021-12-01", date_max = "2022-01-31")[["tmax"]] |> 
  merge(ghcnd_search(stationid = "JA000047759", date_min = "2021-12-01", date_max = "2022-01-31")[["tmin"]], by = c("date", 'id')) |> 
  merge(ghcnd_search(stationid = "JA000047759", date_min = "2021-12-01", date_max = "2022-01-31")[["prcp"]], by = c("date", 'id')) |> 
  select(date, tmax, tmin, prcp)|> 
  mutate(tmax = tmax / 10, tmin = tmin / 10) |> 
  mutate(tmin = case_when(is.na(tmin) == TRUE ~ mean(tmin, na.rm = TRUE),
                          TRUE ~ tmin)) |> 
  rbind(kyoto_extra) |> 
  mutate(city = "Kyoto")

kyoto_weather

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/JA000047759.dly

date created (size, mb): 2022-02-21 00:47:14 (1.666)

file min/max dates: 1945-10-01 / 2022-02-28

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/JA000047759.dly

date created (size, mb): 2022-02-21 00:47:14 (1.666)

file min/max dates: 1945-10-01 / 2022-02-28

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/JA000047759.dly

date created (size, mb): 2022-02-21 00:47:14 (1.666)

file min/max dates: 1945-10-01 / 2022-02-28



date,tmax,tmin,prcp,city
<date>,<dbl>,<dbl>,<dbl>,<chr>
2021-12-01,12.8,2.186111,320,Kyoto
2021-12-02,12.1,2.800000,0,Kyoto
2021-12-03,13.3,3.300000,0,Kyoto
2021-12-04,12.2,2.186111,0,Kyoto
2021-12-05,9.6,4.700000,0,Kyoto
2021-12-06,14.2,4.300000,0,Kyoto
2021-12-07,11.6,10.200000,300,Kyoto
2021-12-08,12.6,2.186111,41,Kyoto
2021-12-09,15.8,6.300000,0,Kyoto


In [112]:
liestal_weather <- ghcnd_search(stationid = "GME00127786", date_min = "2021-12-01", date_max = "2022-01-31")[["tmax"]] |> 
  merge(ghcnd_search(stationid = "GME00127786", date_min = "2021-12-01", date_max = "2022-01-31")[["tmin"]], by = c("date", 'id')) |> 
  merge(ghcnd_search(stationid = "GME00127786", date_min = "2021-12-01", date_max = "2022-01-31")[["prcp"]], by = c("date", 'id')) |> 
  select(date, tmax, tmin, prcp)|> 
  mutate(tmax = tmax / 10, tmin = tmin / 10) |> 
  rbind(liestal_extra) |> 
  mutate(city = "Liestal")

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/GME00127786.dly

date created (size, mb): 2022-02-21 03:07:32 (1.559)

file min/max dates: 1953-09-01 / 2021-12-31

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/GME00127786.dly

date created (size, mb): 2022-02-21 03:07:32 (1.559)

file min/max dates: 1953-09-01 / 2021-12-31

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/GME00127786.dly

date created (size, mb): 2022-02-21 03:07:32 (1.559)

file min/max dates: 1953-09-01 / 2021-12-31



In [113]:
dc_weather <- dc_extra |> 
  slice(31:n()) |> 
  mutate(city = "Washingtondc")

In [114]:
van_weather <- ghcnd_search(stationid = "CA001108395", date_min = "2021-12-01", date_max = "2022-01-31")[["tmax"]] |> 
  merge(ghcnd_search(stationid = "CA001108395", date_min = "2021-12-01", date_max = "2022-01-31")[["tmin"]], by = c("date", 'id')) |> 
  merge(ghcnd_search(stationid = "CA001108395", date_min = "2021-12-01", date_max = "2022-01-31")[["prcp"]], by = c("date", 'id')) |> 
  select(date, tmax, tmin, prcp)|> 
  mutate(tmax = tmax / 10, tmin = tmin / 10) |> 
  rbind(van_extra) |> 
  mutate(city = "Vancouver")

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/CA001108395.dly

date created (size, mb): 2022-02-21 20:12:58 (1.283)

file min/max dates: 1957-01-01 / 2022-02-28

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/CA001108395.dly

date created (size, mb): 2022-02-21 20:12:58 (1.283)

file min/max dates: 1957-01-01 / 2022-02-28

using cached file: C:\Users\Maoli\AppData\Local/Cache/R/noaa_ghcnd/CA001108395.dly

date created (size, mb): 2022-02-21 20:12:58 (1.283)

file min/max dates: 1957-01-01 / 2022-02-28



In [213]:
# create extrapolated tmax, tmin and prcp by year

extra_weather <- rbind(kyoto_weather, liestal_weather, dc_weather, van_weather)
extra_weather_year <- extra_weather |> 
  mutate(year = year(date), month = month(date)) |> 
  filter(year >= 2022) |> 
  group_by(city, year) |> 
  summarise(tmax = mean(tmax, na.rm = TRUE),
            tmin = mean(tmin, na.rm = TRUE),
            prcp = mean(prcp, na.rm = TRUE))

extra_weather_year

`summarise()` has grouped output by 'city'. You can override using the `.groups` argument.



city,year,tmax,tmin,prcp
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Kyoto,2022,21.850329,12.3418795,43.62101
Kyoto,2023,21.850329,12.2894016,50.44220
Kyoto,2024,21.805929,12.2513676,50.45202
Kyoto,2025,21.850329,12.2894016,50.45204
Kyoto,2026,21.850329,12.2894016,50.45204
Kyoto,2027,21.850329,12.2894016,50.45204
Kyoto,2028,21.816312,12.2533649,50.45204
Kyoto,2029,21.850329,12.2894016,50.45204
Kyoto,2030,21.850329,12.2894016,50.45204


In [214]:
# create extrapolated agdd data

tbase <- 0
dflist <- list(kyoto_weather, liestal_weather, dc_weather, van_weather)

for (i in c(1, 2, 3, 4)) {
    df <- dflist[[i]] |> 
      mutate(year = year(date), month = month(date)) |> 
      mutate(gdd = (tmax + tmin) / 2 - tbase) |> 
      mutate(gdd = case_when(gdd < 0 ~ 0,
                             TRUE ~ gdd)) |> 
      group_by(city, year, month) |> 
      summarise(agdd = sum(gdd, na.rm = TRUE)) |> 
      ungroup() |> 
      mutate(mu = case_when(month %in% c(12, 1, 2) ~ "yes",
                            TRUE ~ "no")) |> 
      mutate(lag_agdd = lag(agdd, 1),
             lag_mu = lag(mu, 1)) |> 
      filter(lag_mu == "yes") |> 
      group_by(city, year, lag_mu) |>
      summarise(agdd_winter = sum(lag_agdd, na.rm = TRUE)) |> 
      select(-lag_mu)
    
    extra_weather_year <- left_join(extra_weather_year, df,  by = c("year", "city"))
    
    if ("agdd_winter.x" %in% names(extra_weather_year)){
        extra_weather_year <- extra_weather_year |> 
            mutate(agdd_winter = coalesce(agdd_winter.x, agdd_winter.y)) |> 
            select(-agdd_winter.x, -agdd_winter.y)
        }
    
}

extra_weather_year <- extra_weather_year |> filter(year < 2032)



`summarise()` has grouped output by 'city', 'year'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'city', 'year'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'city', 'year'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'city', 'year'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'city', 'year'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'city', 'year'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'city', 'year'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'city', 'year'. You can override using the `.groups` argument.



In [215]:
extra_weather_year

city,year,tmax,tmin,prcp,agdd_winter
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Kyoto,2022,21.85033,12.341880,43.62101,582.8057
Kyoto,2023,21.85033,12.289402,50.44220,577.5612
Kyoto,2024,21.80593,12.251368,50.45202,590.8612
Kyoto,2025,21.85033,12.289402,50.45204,582.3021
Kyoto,2026,21.85033,12.289402,50.45204,582.3021
Kyoto,2027,21.85033,12.289402,50.45204,582.3021
Kyoto,2028,21.81631,12.253365,50.45204,593.0470
Kyoto,2029,21.85033,12.289402,50.45204,585.5970
Kyoto,2030,21.85033,12.289402,50.45204,585.5970


In [220]:
# extrapolate of weather data of winter

dflist <- list(kyoto_weather, liestal_weather, dc_weather, van_weather)

for (i in c(1, 2, 3, 4)) {
    df <- dflist[[i]] |> 
      mutate(year = year(date), month = month(date)) |> 
      group_by(city, year, month) |> 
      summarise(tmax = mean(tmax, na.rm = TRUE),
                tmin = mean(tmin, na.rm = TRUE),
                   prcp = mean(prcp, na.rm = TRUE)) |> 
      ungroup() |> 
      mutate(mu = case_when(month %in% c(12, 1, 2) ~ "yes",
                                TRUE ~ "no")) |> 
      mutate(lag_tmax = lag(tmax, 1),
             lag_tmin = lag(tmin, 1),
             lag_prcp = lag(prcp, 1),
             lag_mu = lag(mu, 1)) |> 
      filter(lag_mu == "yes") |> 
      group_by(city, year, lag_mu) |>
      summarise(tmax_winter = mean(lag_tmax, na.rm = TRUE),
                tmin_winter = mean(lag_tmin, na.rm = TRUE),
                prcp_winter = mean(lag_prcp, na.rm = TRUE)) |> 
      select(-lag_mu)
    
    extra_weather_year <- left_join(extra_weather_year, df,  by = c("year", "city"))
    
    if ("tmax_winter.x" %in% names(extra_weather_year)){
        extra_weather_year <- extra_weather_year |> 
            mutate(tmax_winter = coalesce(tmax_winter.x, tmax_winter.y),
                   tmin_winter = coalesce(tmin_winter.x, tmin_winter.y),
                   prcp_winter = coalesce(prcp_winter.x, prcp_winter.y)) |> 
            select(-tmax_winter.x, -tmax_winter.y,
                   -tmin_winter.x, -tmin_winter.y,
                   -prcp_winter.x, -prcp_winter.y)
        }
    
}

extra_weather_year <- extra_weather_year |> filter(year < 2032)


extra_weather_year

`summarise()` has grouped output by 'city', 'year'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'city', 'year'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'city', 'year'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'city', 'year'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'city', 'year'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'city', 'year'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'city', 'year'. You can override using the `.groups` argument.

`summarise()` has grouped output by 'city', 'year'. You can override using the `.groups` argument.



city,year,tmax,tmin,prcp,agdd_winter,tmax_winter,tmin_winter,prcp_winter
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Kyoto,2022,21.85033,12.341880,43.62101,582.8057,10.903068,2.1069696,25.12561
Kyoto,2023,21.85033,12.289402,50.44220,577.5612,10.903068,1.9941854,50.39764
Kyoto,2024,21.80593,12.251368,50.45202,590.8612,10.984386,2.0487982,50.45195
Kyoto,2025,21.85033,12.289402,50.45204,582.3021,10.990196,2.0266361,50.45204
Kyoto,2026,21.85033,12.289402,50.45204,582.3021,10.990196,2.0266361,50.45204
Kyoto,2027,21.85033,12.289402,50.45204,582.3021,10.990196,2.0266361,50.45204
Kyoto,2028,21.81631,12.253365,50.45204,593.0470,11.002154,2.0855900,50.45204
Kyoto,2029,21.85033,12.289402,50.45204,585.5970,11.013712,2.0819503,50.45204
Kyoto,2030,21.85033,12.289402,50.45204,585.5970,11.013712,2.0819503,50.45204


In [70]:
# extrapolate co2 data and create predict dataframe

co2 <- read.csv("raw/co-emissions-per-capita.csv")

In [71]:
co2 <- co2 |> 
    filter(Entity %in% c("Japan", "Canada", "United States", "Switzerland")) |> 
    mutate(co2_percapita = Annual.CO2.emissions..per.capita.,
           country = Entity,
           year = Year) |> 
    select(co2_percapita, country, year) |> 
    mutate(city = case_when(country == "Canada" ~"Vancouver",
                            country == "Japan" ~"Kyoto",
                            country == "United States" ~"Washingtondc",
                            country ==  "Switzerland" ~"Liestal"),
           lat = case_when(country == "Canada" ~ 49.2827,
                           country == "Japan" ~ 35.0147,
                           country == "United States" ~ 38.8854,
                           country ==  "Switzerland" ~ 47.4814),
           long = case_when(country == "Canada" ~ -123.1207,
                           country == "Japan" ~ 135.6761,
                           country == "United States" ~ -77.0386,
                           country ==  "Switzerland" ~ 47.4814),
           alt = case_when(country == "Canada" ~ 4.3,
                           country == "Japan" ~ 25.2,
                           country == "United States" ~ 0,
                           country ==  "Switzerland" ~ 350)) |> 
    filter(year >= 1800)

In [55]:
arima_kyoto <- auto.arima(ts((co2 |>  filter(city =="Kyoto"))$co2_percapita, freq = 1))
arima_dc <- auto.arima(ts((co2 |>  filter(city =="Washingtondc"))$co2_percapita, freq = 1))
arima_liestal <- auto.arima(ts((co2 |>  filter(city =="Liestal"))$co2_percapita, freq = 1))
arima_van <- auto.arima(ts((co2 |>  filter(city =="Vancouver"))$co2_percapita, freq = 1))

In [57]:
kyoto_co2hat <- forecast(arima_kyoto, h = 11)
dc_co2hat <- forecast(arima_dc, h = 11)
liestal_co2hat <- forecast(arima_liestal, h = 11)
van_co2hat <- forecast(arima_van, h = 11)

In [84]:
co2_extra <- tibble(
    co2_percapita = c(kyoto_co2hat$mean, dc_co2hat$mean, liestal_co2hat$mean, van_co2hat$mean),
    city = c(rep("Kyoto", 11), rep("Washingtondc", 11) ,rep("Liestal", 11) , rep("Vancouver", 11)),
    year = c(rep(seq(2021, 2031), 4))
)

In [85]:
co2_extra <- co2_extra |> 
  left_join((co2 |> select(-year, -co2_percapita)), by = "city") |> 
  distinct()

In [86]:
co2_extra

co2_percapita,city,year,country,lat,long,alt
<dbl>,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>
7.907034,Kyoto,2021,Japan,35.0147,135.6761,25.2
7.720685,Kyoto,2022,Japan,35.0147,135.6761,25.2
7.577700,Kyoto,2023,Japan,35.0147,135.6761,25.2
7.467988,Kyoto,2024,Japan,35.0147,135.6761,25.2
7.383807,Kyoto,2025,Japan,35.0147,135.6761,25.2
7.319216,Kyoto,2026,Japan,35.0147,135.6761,25.2
7.269655,Kyoto,2027,Japan,35.0147,135.6761,25.2
7.231627,Kyoto,2028,Japan,35.0147,135.6761,25.2
7.202449,Kyoto,2029,Japan,35.0147,135.6761,25.2


In [ ]:
prediction <- merge(co2_extra, )